First, we load data into memory and perform a simple statistical analysis on the data.

In [1]:
set.seed(666)
data <- load(url("https://github.com/zhentaoshi/Econ5821/raw/main/data_example/dataset_inf.Rdata"))
summary(cpi)
summary(ppi)

     month             CPI       
 Min.   :  1.00   Min.   : 98.2  
 1st Qu.: 42.75   1st Qu.:101.5  
 Median : 84.50   Median :102.3  
 Mean   : 84.50   Mean   :102.7  
 3rd Qu.:126.25   3rd Qu.:103.6  
 Max.   :168.00   Max.   :108.7  

     month             PPI        
 Min.   :  1.00   Min.   : 91.80  
 1st Qu.: 42.75   1st Qu.: 97.92  
 Median : 84.50   Median :102.46  
 Mean   : 84.50   Mean   :101.46  
 3rd Qu.:126.25   3rd Qu.:105.22  
 Max.   :168.00   Max.   :110.10  

Then, we join the features, cpi and ppi together using \`month\`. Since we can only use features in the past to forecast current ppi or cpi, we put the cpi or ppi with the features of last month together.

In [2]:
cpi$month = cpi$month - 1
ppi$month = ppi$month - 1
data <- merge(X, cpi, by="month")
data <- merge(data, ppi, by="month")

We will use r square as our criteria.

We first try XGBoost.

As XGBoost using specific data format, we must transform our data to DMatrix. And at the same time, we split the data into training data and testing data by the ratio 8:2.

In [3]:
library(xgboost)
library(Matrix)

train_index = sample(nrow(data),0.8*nrow(data))
train_data = data[train_index,]
test_data = data[-train_index,]
trainX = data.matrix(train_data[,c(2:152,154)])
trainX = Matrix(trainX,sparse=T)
trainY = train_data[,153]
data = list(x = trainX, y = trainY )
dtrain = xgb.DMatrix(data = data$x, label = data$y)

testX = data.matrix(test_data[,c(2:152,154)])
testX = Matrix(testX,sparse=T)
testY = test_data[,153]
data = list(x = testX, y = testY )
dtest = xgb.DMatrix(data = data$x, label = data$y)

Warning message:
"package 'xgboost' was built under R version 4.2.3"


After preparing the training data and testing data, we start to train the XGBoost model， we tried different combinations of hyper-parameters. Lastly, we can get the best model below.

In [4]:
xgb <- xgboost(data = dtrain,max_depth=4,eta=0.1, nround=246)
#test data (oos r2)
pre_xgb = predict(xgb,newdata = dtest)
rss = var(testY - pre_xgb)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_xgb = predict(xgb,newdata = dtrain)
rss = var(trainY - pre_xgb)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

[1]	train-rmse:92.094273 
[2]	train-rmse:82.958074 
[3]	train-rmse:74.729167 
[4]	train-rmse:67.317563 
[5]	train-rmse:60.642196 
[6]	train-rmse:54.630061 
[7]	train-rmse:49.215405 
[8]	train-rmse:44.339010 
[9]	train-rmse:39.947541 
[10]	train-rmse:35.992971 
[11]	train-rmse:32.432041 
[12]	train-rmse:29.225809 
[13]	train-rmse:26.339212 
[14]	train-rmse:23.740668 
[15]	train-rmse:21.401759 
[16]	train-rmse:19.296900 
[17]	train-rmse:17.403064 
[18]	train-rmse:15.699530 
[19]	train-rmse:14.164704 
[20]	train-rmse:12.781635 
[21]	train-rmse:11.535601 
[22]	train-rmse:10.411638 
[23]	train-rmse:9.398562 
[24]	train-rmse:8.485733 
[25]	train-rmse:7.662020 
[26]	train-rmse:6.919575 
[27]	train-rmse:6.250763 
[28]	train-rmse:5.647551 
[29]	train-rmse:5.103761 
[30]	train-rmse:4.614569 
[31]	train-rmse:4.172623 
[32]	train-rmse:3.774740 
[33]	train-rmse:3.415618 
[34]	train-rmse:3.093058 
[35]	train-rmse:2.801996 
[36]	train-rmse:2.539242 
[37]	train-rmse:2.302804 
[38]	train-rmse:2.089932 

[1] 0.9406761

[1] 0.9999999

Then we use random forest.

In [5]:
library(randomForest)

rf = randomForest(CPI ~ . ,ntree=100, data = train_data)
#test data (oos r2)
pre_rf = predict(rf,newdata = test_data)
rss = var(testY - pre_rf)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_rf = predict(rf,newdata = train_data)
rss = var(trainY - pre_rf)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.



[1] 0.9263844

[1] 0.9818031

kNN

In [6]:
library(caret)
control = trainControl(method = 'cv',number = 10)
kNN = train(CPI~.,train_data,
               method = 'knn',
               trControl = control,
               tuneLength = 5)
#test data (oos r2)
pre_kNN = predict(kNN,newdata = test_data)
rss = var(testY - pre_kNN)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_kNN = predict(kNN,newdata = train_data)
rss = var(trainY - pre_kNN)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

Warning message:
"package 'caret' was built under R version 4.2.3"
Loading required package: ggplot2


Attaching package: 'ggplot2'


The following object is masked from 'package:randomForest':

    margin


Loading required package: lattice



[1] 0.6479266

[1] 0.7341877

decision tree

In [7]:
library(rpart)
DT = rpart(CPI~., train_data)
#test data (oos r2)
pre_DT = predict(DT, newdata = test_data)
rss = var(testY - pre_DT)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_DT = predict(DT, newdata = train_data)
rss = var(trainY - pre_DT)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

[1] 0.8574528

[1] 0.9145992

lasso

In [8]:
library(glmnet)
x = trainX
y = trainY
cv_lasso = cv.glmnet(x, y)
lasso_result = glmnet(x, y, lambda = cv_lasso$lambda.min)
#test data (oos r2)
pre_lasso = predict(lasso_result, newx = testX)
rss = var(testY - pre_lasso)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_lasso = predict(lasso_result, newx = trainX)
rss = var(trainY - pre_lasso)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

Loaded glmnet 4.1-6



,s0
s0,0.9559302


,s0
s0,0.9588158


neural network

In [16]:
library(neuralnet)
nn = neuralnet(CPI ~ ., train_data,hidden = 5)
#test data (oos r2)
pre_nn = predict(nn, newdata = test_data)
rss = var(testY - pre_nn)
tss = var(testY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_nn = predict(nn, newdata = train_data)
rss = var(trainY - pre_nn)
tss = var(trainY)
is_r2 = 1-rss/tss
is_r2

Warning message:
"package 'neuralnet' was built under R version 4.2.3"


0


0.05966124


By comparing the results of CPI above, we can see that the xgboost has the best r2 in the train data set, but lasso has the best r2 in the test data set.

As for ppi, we first use XGBoost.

In [9]:
PPItrainX = data.matrix(train_data[,c(2:153)])
PPItrainX = Matrix(trainX,sparse=T)
PPItrainY = train_data[,154]
data = list(x = PPItrainX, y = PPItrainY )
PPIdtrain = xgb.DMatrix(data = data$x, label = data$y)

PPItestX = data.matrix(test_data[,c(2:153)])
PPItestX = Matrix(testX,sparse=T)
PPItestY = test_data[,154]
data = list(x = PPItestX, y = PPItestY )
PPIdtest = xgb.DMatrix(data = data$x, label = data$y)

In [10]:
PPIxgb <- xgboost(data = PPIdtrain, max_depth = 12, eta = 0.3,nround=63)
#test data (oos r2)
pre_PPIxgb = predict(PPIxgb,newdata = PPIdtest)
rss = var(PPItestY - pre_PPIxgb)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_PPIxgb = predict(PPIxgb,newdata = PPIdtrain)
rss = var(PPItrainY - pre_PPIxgb)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

[1]	train-rmse:70.961785 
[2]	train-rmse:49.932317 
[3]	train-rmse:35.206667 
[4]	train-rmse:24.841854 
[5]	train-rmse:17.555903 
[6]	train-rmse:12.430062 
[7]	train-rmse:8.802840 
[8]	train-rmse:6.243671 
[9]	train-rmse:4.440135 
[10]	train-rmse:3.172052 
[11]	train-rmse:2.273734 
[12]	train-rmse:1.639541 
[13]	train-rmse:1.193533 
[14]	train-rmse:0.878736 
[15]	train-rmse:0.656639 
[16]	train-rmse:0.497698 
[17]	train-rmse:0.379529 
[18]	train-rmse:0.293322 
[19]	train-rmse:0.229822 
[20]	train-rmse:0.182401 
[21]	train-rmse:0.146134 
[22]	train-rmse:0.118486 
[23]	train-rmse:0.096670 
[24]	train-rmse:0.079631 
[25]	train-rmse:0.065997 
[26]	train-rmse:0.054946 
[27]	train-rmse:0.045830 
[28]	train-rmse:0.038377 
[29]	train-rmse:0.032244 
[30]	train-rmse:0.027134 
[31]	train-rmse:0.022889 
[32]	train-rmse:0.019333 
[33]	train-rmse:0.016334 
[34]	train-rmse:0.013813 
[35]	train-rmse:0.011703 
[36]	train-rmse:0.009918 
[37]	train-rmse:0.008409 
[38]	train-rmse:0.007126 
[39]	train-rmse

[1] 0.9896343

[1] 1

randomForest

In [11]:
rf = randomForest(PPI ~ . , data = train_data,ntree = 500)

#test data (oos r2)
pre_rf = predict(rf,newdata = test_data)
rss = var(PPItestY - pre_rf)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_rf = predict(rf,newdata = train_data)
rss = var(PPItrainY - pre_rf)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

[1] 0.9538588

[1] 0.9946857

kNN

In [12]:
control = trainControl(method = 'cv',number = 10)
kNN = train(PPI~.,train_data,
               method = 'knn',
               trControl = control,
               tuneLength = 5)
#test data (oos r2)
pre_kNN = predict(kNN,newdata = test_data)
rss = var(PPItestY - pre_kNN)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_kNN = predict(kNN,newdata = train_data)
rss = var(PPItrainY - pre_kNN)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

[1] 0.7811055

[1] 0.8665784

decision tree

In [13]:
DT = rpart(PPI~., train_data)
#test data (oos r2)
pre_DT = predict(DT, newdata = test_data)
rss = var(PPItestY - pre_DT)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_DT = predict(DT, newdata = train_data)
rss = var(PPItrainY - pre_DT)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

[1] 0.9220038

[1] 0.9506028

lasso

In [14]:
x = PPItrainX
y = PPItrainY
cv_lasso = cv.glmnet(x, y)
lasso_result = glmnet(x, y, lambda = cv_lasso$lambda.min)
#test data (oos r2)
pre_lasso = predict(lasso_result, newx = PPItestX)
rss = var(PPItestY - pre_lasso)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_lasso = predict(lasso_result, newx = PPItrainX)
rss = var(PPItrainY - pre_lasso)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

,s0
s0,0.9990674


,s0
s0,0.9990674


neural network

In [17]:
nn = neuralnet(PPI ~ ., train_data,hidden = 20)
#test data (oos r2)
pre_nn = predict(nn, newdata = test_data)
rss = var(PPItestY - pre_nn)
tss = var(PPItestY)
oos_r2 = 1-rss/tss
oos_r2
#train data (is r2)
pre_nn = predict(nn, newdata = train_data)
rss = var(PPItrainY - pre_nn)
tss = var(PPItrainY)
is_r2 = 1-rss/tss
is_r2

-0.1100742


0.01697305


By comparing the results of PPI above, we can see that the xgboost has the best r2 in the train data set, but lasso has the best r2 in the test data set.

We can now use lasso to make the prediction of both CPI and PPI.